# Overpy Folium Mapping Toolkit
This is meant to be used help make maps with Python using the Overpy and Folium packages.

In [2]:
import pandas as pd
import requests
import json
import folium
from folium.features import DivIcon
import overpy
import geopandas as gpd
import contextily as cx
from copy import deepcopy
import geopy.distance
from shapely.geometry import Polygon, Point, LineString

import pickle
import webbrowser

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
from math import atan2, degrees

api = overpy.Overpass()

In [3]:
# load map tiles   
with open("tiles.json") as f:
    tiles = json.load(f)
tiles = {name: folium.TileLayer(tiles = data["url"],
                                attr = data["attr"], 
                                name = name, 
                                overlay = False, 
                                control = True, 
                                show = True
                               ) for name, data in tiles.items()}
print(f"tiles: {list(tiles.keys())}")


# Load nation lakes coords   
with open("nation_lakes.json") as f:
    lakes = json.load(f)
print("\nlakes:")
for i,lake in enumerate(lakes):
    print(i,lake["name"])


tiles: ['Stamen Toner', 'Stamen Toner Lite', 'Stamen Terrain', 'Stamen Watercolor', 'Google Maps', 'Google Satellite', 'Google Satellite Hybrid', 'Esri Satellite', 'Cartodb Positron']

lakes:
0 Tsayta Lake
1 Indata Lake
2 Tchentlo Lake
3 Chuchi Lake


# Get Overpy Result object
Query Overpass using Overpy to get `overpy.Result object`. This has all the info but is not super easy to use, because the nodes, ways, and relations are buried in lists and do not have complete information. 


In [4]:
def get_nwr_in_bbox(corners, desired_tags = {}, buffer = 0):
    """Gets all Nodes, Ways, Relations inside bounding box, optionally with specified tag
    PARAMETERS:
        corners(tuple):      limits of bounding box (south,west,north,east)
        desired_tags(dict):  tags to retrieve within bbox (default = {}, returns all Nodes, Ways, Relations)
                     eg {'tourism': ['camp_site'], 'name': ['Ahdatay', 'Pine Point']}
    RETURNS:
        Overpy.Result object
    """
    buffer = (-buffer, -buffer, buffer, buffer)
    corners = tuple([c+b for c,b in zip(corners, buffer)])
    
    query = build_query(corners, desired_tags)
    result = api.query(query)
    
    return result


def build_query(corners, desired_tags = None):
    query = f"""
nwr{corners}->.all;
(
nwr.all;
);
out center;
    """
    if desired_tags!=None:
        tag_str = ""
        for key, values in desired_tags.items():
            for value in values:
                tag_str += f'nwr.all["{key}"~"{value}"];\n'
        query = query.replace("nwr.all;", tag_str)
    return query


In [5]:
place = lakes[1]
print(place)
corners = (place["south"],place["west"],place["north"],place["east"])

result_file = "tsayta-lake-result.pkl"

desired_tags = {
    # 'name': ['Tsayta Lake', 'Indata Lake', 'Tchentlo Lake', 'Chuchi Lake', 'Nation Lakes Provincial Park', 'Nation River'],
    'leisure': ['park','nature_reserve'],
    'tourism': ['camp_site'],
    'natural': ['peak', 'mountain_range', 'volcano', 'spring'],
    'highway': ['motorway', 'trunk','primary','secondary','tertiary','unclassified','track','road','path', 'footway'],
    'water':['lake', 'river'],
    'name': ['Fall-Tsayta Forest Service Road']
}

# Download from internet
result = get_nwr_in_bbox(corners, desired_tags, buffer = 0.05)

# # save result to file
# with open(result_file, "wb") as f:
#     pickle.dump(result, f)
# print(f"Overpass result saved to: {result_file}")

# # load result from file
# with open(result_file, "rb") as f:
#     result = pickle.load(f)

print(place["name"],":")
print(len(result.nodes), "nodes")
print(len(result.ways), "ways")
print(len(result.relations), "relations")


{'name': 'Indata Lake', 'north': 55.3925, 'west': -125.32, 'south': 55.292, 'east': -125.2175, 'lat': 55.34225, 'lon': -125.26875}
Indata Lake :
9 nodes
27 ways
4 relations


# OfmtResult Class
Used for parsing Overpy Result objects and preparing them for plotting.

In [6]:


class OfmtNode:
    def __init__(self, node_id, result):
        node = result.get_node(node_id, resolve_missing = True)
        self.id = node_id
        self.lat = float(node.lat)
        self.lon = float(node.lon)
        self.name = node.tags["name"] if "name" in node.tags else None
        if "name" in node.tags:
            del node.tags["name"]
        self.tags = node.tags
    
    def __str__(self):
        return f"id:{self.id}, name:{self.name}, lat:{self.lat}, lat:{self.lon}, tags:{self.tags}"
        
        
class OfmtWay:
    def __init__(self, way_id, result):
        self.id = way_id
        way = result.get_way(way_id, resolve_missing = True)
        self.coords = [np.array([float(node.lat), float(node.lon)]) for node in way.get_nodes(resolve_missing = True)]
        self.name = way.tags["name"] if "name" in way.tags else None
        if "name" in way.tags:
            del way.tags["name"] 
        self.tags = way.tags
    
    def __str__(self):
        return f"id:{self.id}, name:{self.name}, {len(self.coords)} nodes, tags:{self.tags}"
    
    
class OfmtRelation:
    def __init__(self, relation_id, result):
        self.id = relation_id
        relation = result.get_relation(relation_id, resolve_missing = True)
        relation_ways = [member for member in relation.members if type(member)==overpy.RelationWay]
        self.ways = []
        self.nodes = []
        for member in relation.members:
            if type(member)==overpy.RelationWay:
                self.ways.append(OfmtWay(member.ref, result))
            elif type(member)==overpy.RelationNode:
                self.nodes.append(OfmtNode(member.ref, result))
        self.name = relation.tags["name"] if "name" in relation.tags else None
        if "name" in relation.tags:
            del relation.tags["name"] 
        self.tags = relation.tags
    
    def __str__(self):
        return f"id:{self.id}, name:{self.name}, {len(self.nodes)} nodes, {len(self.ways)} ways, tags:{self.tags}"

In [7]:
class OfmtResult:
    def __init__(self, result=None):
        self.result = deepcopy(result)
        self.nodes = self.parse_nodes()
        self.ways = self.parse_ways()
        self.relations = self.parse_relations()
        return
    
    def __str__(self):
        return f"{len(self.nodes)} nodes, {len(self.ways)} ways, {len(self.relations)} relations"
    
    def parse_nodes(self):
        if self.result.nodes == None:
            print("The result contains no nodes") 
        else:
            nodes = []
            for node in self.result.nodes:
                nodes.append(OfmtNode(node.id, self.result))
        return nodes

    def parse_ways(self):
        if self.result.ways == None:
            print("The result contains no ways")  
        else:
            ways = []
            for way in self.result.get_ways():
                ways.append(OfmtWay(way.id, self.result)) 
        return ways
    
    def parse_relations(self):
        if self.result.relations == None:
            print("The result contains no ways")  
        else:
            relations = []
            for relation in self.result.get_relations():
                relations.append(OfmtRelation(relation.id, self.result)) 
        return relations

    def get_unique_tags(self):
        """Finds every unique tag in list of Nodes, Ways, or Relations
        PARAMETERS:
        
            items(list):  list of Nodes, Ways, or Relations from Overpass.Result, eg. Overpass.Result.nodes
        RETURNS:
            dict:        eg.{key1: [value1, value2], key2: [value3]}
        """
        tags = {}
        collections = [self.nodes, self.ways, self.relations]
        for collection in collections:
            for item in collection:
                for key,value in item.tags.items():
                    if key not in tags:
                        tags[key] = [value]
                    elif value not in tags[key]:
                        tags[key].append(value)
        return tags

In [8]:
res = OfmtResult(result)
print(res)

9 nodes, 27 ways, 4 relations


In [9]:
res.get_unique_tags()

{'tourism': ['camp_site'],
 'natural': ['peak', 'water'],
 'source': ['NRCan-CanVec-10.0',
  'DataBC TANTALIS - Parks, Ecological Reserves, and Protected Areas',
  'NRCan-CanVec-10.0;MAXAR 2018'],
 'highway': ['tertiary', 'unclassified', 'track'],
 'surface': ['unpaved'],
 'tracktype': ['grade5', 'grade4'],
 'ford': ['yes'],
 'bridge': ['yes'],
 'water': ['river', 'lake'],
 'layer': ['1'],
 'boundary': ['national_park'],
 'leisure': ['nature_reserve'],
 'type': ['multipolygon'],
 'wikidata': ['Q6970045'],
 'wikipedia': ['en:Nation Lakes Provincial Park']}

In [10]:
#     def make_node_df(self):
#         """Retrieves relevant node information on nodes from container
#         PARAMETERS:
#             container(Overpy.X object): Object that contains nodes. One of: (Overpy.Result, Overpy.Way, Overpy.Relation, Overpy.RelationWay)
#         RETURNS:
#             geodataframe
#         """
#         nodes = []
#         for node_id, node in self.nodes.items():
#                 nodes.append({
#                     "id": node_id,
#                     "name": node["tags"]["name"] if "name" in node["tags"] else np.nan,
#                     "lat": float(node["lat"]),
#                     "lon": float(node["lon"]),
#                     "tags": node["tags"]
#                 })
#         df = pd.DataFrame(nodes)    
#         df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326") 
#         self.node_df = df
#         return df
    
    
#     def make_way_df(self):
#         """Retrieves relevant way information from container, including all nodes
#         PARAMETERS:
#             container(Overpy.Result object): Object that contains ways
#         RETURNS:
#             dict:      keys = way.id, values = nodes,tags
#         """ 
#         ways = []
#         for way_id,way in self.ways.items():
#             nodes = way["nodes"]
#             linestring = [(float(node["lon"]), float(node["lat"])) for node in nodes.values()]
#             ways.append({
#                 "id": way_id,
#                 "name": way["tags"]["name"] if "name" in way["tags"] else np.nan,
#                 "tags": way["tags"],
#                 "geometry": LineString(linestring)
#             })
            
#         df = gpd.GeoDataFrame(ways, geometry="geometry")
#         return df


# OfmtMap Class

In [11]:
class OfmtMap:
    def __init__(self, name, north, south, east, west, tile, map_args = {}):
        self.name = name
        self.north,self.south,self.east,self.west = (north, south, east, west)
        self.centre = self.calculate_centre()
        self.tile= tile
        
        if map_args == {}:
            tile = self.tile
            tile.overlay = False
            tile.show = True
            map_args = dict(
                location = self.centre, 
                min_zoom = 8, 
                max_zoom = 22,
                zoom_start = cx.tile._calculate_zoom(self.west, self.south, self.east, self.north)-2,
                zoomSnap = 0.25,
                zoomDelta = 0.25,
                tiles = self.tile,
                control_scale = True,
                zoom_control = False,
                boxZoom = True,
            )
        self.map_args = map_args
        self.map = None
        
    def make_map(self):
        m = folium.Map(**self.map_args,)
        # m.add_child(folium.LatLngPopup())
        self.map = m
        return
        
        
    def calculate_centre(self):
        lat = (self.north + self.south)/2
        lon = (self.east + self.west)/2
        return lat, lon
          
        
    def update_map_args(self, new_map_args):
        self.map_args = self.map_args | new_map_args
        return
    
    
    def add_node(self, node, icon = None, hover_text = None, **kwargs):
        if icon == None:
            marker = folium.CircleMarker(location=[node.lat, node.lon], tooltip = hover_text, **kwargs)
        else:
            marker = folium.Marker(
                location=[node.lat, node.lon], 
                icon=icon,
                tooltip = hover_text, **kwargs
            )
        marker.add_to(self.map) 
        return
    
    def add_text(self, lat, lon, text, icon_size=(400,10), icon_anchor=(0,0),**kwargs):
        icon=DivIcon(
            icon_size=icon_size, icon_anchor=icon_anchor,
            html=f'<div style="font-size: {kwargs["font_size"]}pt; color:{kwargs["color"]}">{text}</div>'
        )
        marker = folium.map.Marker([lat, lon], icon=icon)
        marker.add_to(self.map)
        return
    
    def add_map_title(self, title, x=0.5, y=0.75, **kwargs):
        
        n,s,e,w = self.north,self.south,self.east,self.west
        lat = s + (n-s)*y
        lon = w + (e-w)*x
        font_size = kwargs["font_size"]
        icon_size = len(title)*10*font_size, 10
        icon_anchor= len(title)*10/2*font_size, 0
        self.add_text(lat, lon, title, icon_size, icon_anchor=(0,0),**kwargs)
        
        
        return
    
    def add_way(self, way, hover_text = None, **style):
        folium.PolyLine(way.coords, # list of (lat,lon) tuples
                        **style,
                        opacity=1,
                        tooltip =  hover_text
                        ).add_to(self.map)
        return


    def add_relation(self, relation, hover_text = None, **style):
        for way in relation.ways:
            self.add_way(way, hover_text = hover_text, **style)
        for node in relation.nodes:
            self.add_way(way, hover_text = hover_text, **style)
        return


In [12]:
# res = OfmtResult(result)
print(res)
res.get_unique_tags()

9 nodes, 27 ways, 4 relations


{'tourism': ['camp_site'],
 'natural': ['peak', 'water'],
 'source': ['NRCan-CanVec-10.0',
  'DataBC TANTALIS - Parks, Ecological Reserves, and Protected Areas',
  'NRCan-CanVec-10.0;MAXAR 2018'],
 'highway': ['tertiary', 'unclassified', 'track'],
 'surface': ['unpaved'],
 'tracktype': ['grade5', 'grade4'],
 'ford': ['yes'],
 'bridge': ['yes'],
 'water': ['river', 'lake'],
 'layer': ['1'],
 'boundary': ['national_park'],
 'leisure': ['nature_reserve'],
 'type': ['multipolygon'],
 'wikidata': ['Q6970045'],
 'wikipedia': ['en:Nation Lakes Provincial Park']}

In [13]:
styles = {
    ("water","lake"):{"color": "blue", "weight": 1},
    ("water","river"):{"color": "blue", "weight": 1},
    ("tourism","camp_site"):{"color": "green", "weight": 1, "radius":8},
    # ("leisure", "nature_reserve"):{"color": "darkgreen", "dash_array":'5, 1', "weight": 1},#, "fill_color": "green", "fill_opacity": 0.2, },
    ("highway","tertiary"):{"color": "grey", "weight": 1},
    ("highway","unclassified"):{"color": "grey", "weight": 1},
    # ("highway","track"):{"color": "grey", "weight": 1},
    # ("highway","path"):{"color": "grey", "weight": 0.5},
    # ("",""):{"color": "", "weight": 1},
}

def get_style(tags, styles):
    style = None
    for key,value in tags.items():
        if (key,value) in styles:
            style = styles[(key,value)]
            break
    return style   

In [15]:
lake = lakes[1]
name = lake["name"]
north, south, east, west = lake["north"],lake["south"],lake["east"],lake["west"]
tile = tiles["Stamen Toner Lite"]

om = OfmtMap(name, north, south, east, west, tile)
om.make_map()

# add campgrounds
node_args = {"color": "green","weight": 1,"radius": 7}

for node in res.nodes:
    style = get_style(node.tags, styles)
    if style != None:
        om.add_node(node, hover_text = f"{node.name if node.name!=None else node.tags}" , **style)

for way in res.ways:
    style = get_style(way.tags, styles)
    if style != None:
        om.add_way(way, hover_text = f"way:{way.name if way.name!=None else way.tags}", **style)

for relation in res.relations:
    style = get_style(relation.tags, styles)
    if style != None:
        om.add_relation(relation, hover_text = f"{relation.name if relation.name!=None else relation.tags}", **style)   

text_args = {"color": "black","font_size": 30}
# om.add_text(lake["lat"], lake["lon"], lake["name"],**text_args)
om.add_map_title(lake["name"], x=0.5, y=1.5, **text_args)
om.map

In [69]:
print(way)

id:43312131, name:Kwanika-Germansen Road, 31 nodes, tags:{'alt_name': 'Old Germansen Lake Road', 'highway': 'unclassified', 'lanes': '2', 'surface': 'unpaved'}


In [88]:
class OfmtSegments():
    def __init__(self, way):
        self.name = way.name
        self.coords = way
        self.segments = self.make_segments(coords)
        
    def __str__(self):
        return f"name={name}, {len(self.segments)} segments"
        
    def make_segments(self, coords):
        cumul_length = 0
        segments = []
        for i,coord in enumerate(coords[:-1]):
            coords_start = coords[i]
            coords_end = coords[i+1]
            dx = coords_end[1] - coords_start[1]
            dy = coords_end[0] - coords_start[0]

            # print(coords_start, coords_end)
            length = geopy.distance.geodesic(coords_start, coords_end).km
            cumul_length += length
            segments.append({
                "start": coords_start, 
                "end": coords_end,
                "length": length,
                "cumul_length": cumul_length,
                "angle": atan2(dy,dx)
            })        
        return segments
    

In [89]:
way = res.ways[2]
segs = OfmtSegments(way)   
print(way)
print(segs)

id:486287333, name:Driftwood Forest Service Road, 357 nodes, tags:{'highway': 'tertiary', 'surface': 'unpaved'}
name=Tsayta Lake, 413 segments


In [91]:
segs.segments[0:2]

[{'start': array([  55.4373058, -125.3068262]),
  'end': array([  55.4375   , -125.3070559]),
  'length': 0.02605447475720882,
  'cumul_length': 0.02605447475720882,
  'angle': 2.4397456941149507},
 {'start': array([  55.4375   , -125.3070559]),
  'end': array([  55.4375   , -125.3064078]),
  'length': 0.041022093841778906,
  'cumul_length': 0.06707656859898772,
  'angle': 0.0}]

In [92]:
om = OfmtMap(name, north, south, east, west, tile)
om.make_map()

# for node in res.nodes:
#     style = get_style(node.tags, styles)
#     om.add_node(node, hover_text = f"{node.name if node.name!=None else node.tags}" , **style)

# for way in res.ways:
#     style = get_style(way.tags, styles)
#     om.add_way(way, hover_text = f"way:{way.name if way.name!=None else way.tags}", **style)

for relation in res.relations:
    style = get_style(relation.tags, styles)
    if style!=None:
        om.add_relation(relation, hover_text = f"{relation.name if relation.name!=None else relation.tags}", **style)   

style = {"color": "black","font_size": 30}
om.add_map_title(lake["name"], x=0.5, y=1.5, **style)
# om.add_text(lake["lat"], lake["lon"], lake["name"],**text_args)

way = res.ways[2]
segs = OfmtSegments(way)
for i,segment in enumerate(segs.segments):
    marker = folium.Marker(
        location = (segment["start"] + segment["end"])/2,
        tooltip=f'{i}: {segment["angle"]}'
    )
    marker.add_to(om.map) 

om.map

In [180]:
# figure out size of div icon TODO: finish 
text = lake["name"]
font_size = 40
pnt_pxl_conversion = 0.75
text_width_pixel = len(text)*font_size*0.75*pnt_pxl_conversion

# Matplotlib/Contextily

In [181]:
import contextily as cx
import pyproj

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe


In [93]:
# these contexts are essentiall different map tile (background) styles. 
# CartoDB.Positron matches the Citysage look best
contexts = [
    cx.providers.OpenStreetMap.Mapnik,
    cx.providers.Esri.WorldImagery,
    cx.providers.Esri.WorldStreetMap,
    cx.providers.CartoDB.Positron,
    cx.providers.CartoDB.PositronOnlyLabels,
    cx.providers.CartoDB.Voyager,
    cx.providers.Stamen.TonerLite
]
context = contexts[6]
context

{'url': 'https://stamen-tiles-{s}.a.ssl.fastly.net/{variant}/{z}/{x}/{y}{r}.{ext}',
 'html_attribution': 'Map tiles by <a href="http://stamen.com">Stamen Design</a>, <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
 'attribution': 'Map tiles by Stamen Design, CC BY 3.0 -- Map data (C) OpenStreetMap contributors',
 'subdomains': 'abcd',
 'min_zoom': 0,
 'max_zoom': 20,
 'variant': 'toner-lite',
 'ext': 'png',
 'name': 'Stamen.TonerLite'}

In [95]:
df = node_df

NameError: name 'node_df' is not defined

In [94]:
name_col = "node_name"
lat_col = "lat"
lon_col = "lon" 
figsize=(10,6)
map_padding_pct = 15
bg_zoom_adjust = -1
bg_alpha = 1
title_font_size_pct = 5
text_font_size_pct = 1
text_offset_x_pct = 0
text_offset_y_pct = -3.5
markersize_pct = 5
alpha=0.3
facecolor = "#8d5998"
edgecolor = "black"
linewidth = 1
fontfamily = "DejaVu Sans"
title = lake["name"]

# convert to geodataframe
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[lon_col], df[lat_col]), crs="WGS84") # start with WGS84=web mercator
gdf = gdf.to_crs(epsg=3857) # project to spherical mercator to match tiles

fig, ax = plt.subplots(figsize = figsize)
# fig.set_dpi(100)
width, height = figsize[0], figsize[1]

# get extents of points to be plotted
x_min, x_max, y_min, y_max = gdf.geometry.x.min(), gdf.geometry.x.max(), gdf.geometry.y.min(), gdf.geometry.y.max()
centre_x, centre_y  = (x_min + x_max)/2 , (y_min + y_max)/2
min_delta = 150     # determined empirically, controls background zoom level if there is just one point (or points very close together)
delta_x = max((x_max - x_min), min_delta)
delta_y = max((y_max - y_min), min_delta)

# adjust the plot extents to match the desired plot size
width, height = figsize
if width>=height:
    delta_x = max(delta_x, delta_y*width/height) * (1+2*map_padding_pct/100)
    delta_y = delta_x*height/width 
else:
    delta_y = max(delta_y, delta_x*height/width) * (1+2*map_padding_pct/100)
    delta_x = delta_y*width/height 
x_min = centre_x - delta_x/2
x_max = centre_x + delta_x/2
y_min = centre_y - delta_y/2
y_max = centre_y + delta_y/2


# convert plot extents into lat,lon for zoom calculation
proj = pyproj.Transformer.from_crs(3857, 4326, always_xy=True)
lon_min, lat_min = proj.transform(x_min, y_min)
lon_max, lat_max = proj.transform(x_max, y_max)
calculated_zoom_level = cx.tile._calculate_zoom(lon_min, lat_min, lon_max, lat_max)     
zoom_level =  calculated_zoom_level + bg_zoom_adjust


# calculate size of fonts, markers, in points etc
px_pt_conversion = 3/4 # weird... turns out that 12 pt font is 16 px tall
pct_pt_conversion_factor =  fig.dpi * height * px_pt_conversion
text_font_size = text_font_size_pct/100 * pct_pt_conversion_factor
title_font_size = title_font_size_pct/100 * pct_pt_conversion_factor
markersize = markersize_pct/100 * pct_pt_conversion_factor
text_offset_y = text_offset_y_pct/100 * pct_pt_conversion_factor
text_offset_x = text_offset_x_pct/100 * pct_pt_conversion_factor

ax = gdf.plot(
    ax = ax, figsize=figsize, alpha=alpha, facecolor = facecolor, edgecolor = edgecolor, 
    linewidth = linewidth, markersize = markersize
)
ax.set(xlim=(centre_x - (delta_x/2), centre_x + (delta_x/2)), 
       ylim=(centre_y - (delta_y/2), centre_y + (delta_y/2)))
ax.set_axis_off() # don't display axes with coordinates

# add background tiles
crs = gdf.crs.to_string()

# cx.add_basemap(ax, source=context, crs=crs, zoom=zoom_level, attribution=False)
cx.add_basemap(ax, source="https://tiles.stadiamaps.com/tiles/stamen_toner_lite/{z}/{x}/{y}{r}.png?api_key=87e81a5a-4136-409e-92dd-10c6a1a3cb9d",
               crs=crs, zoom=zoom_level, attribution=False)
path_effects = None
path_effects = [pe.withStroke(linewidth = 1, foreground="white", alpha=0.9)]

for idx, row in gdf.iterrows():
    plt.annotate(
        text=row["name"],
        xycoords='data',
        xy=(row.geometry.x, row.geometry.y), 
        xytext = (text_offset_x, text_offset_y),
        textcoords='offset points',
        fontfamily=fontfamily,
        horizontalalignment='center', 
        size = text_font_size,
        zorder = 1000,            # ensures the text is on top
        path_effects=path_effects # puts "glow" border around text if needed
    )

title_position_pad_factor = 1.2
title_ypos = 1-title_font_size_pct/100*title_position_pad_factor
plt.title(title, fontdict={'fontsize': title_font_size, 'fontfamily': fontfamily, "verticalalignment": "center"}, 
          y = title_ypos, 
          zorder = 10000,           # ensures the title is on top
          path_effects=path_effects # puts "glow" border around text if needed
         )

# set border around map
bbox = ax.get_tightbbox(fig.canvas.get_renderer())
x0, y0, w, h = bbox.transformed(fig.transFigure.inverted()).bounds
xpad = 0.03 * w
ypad = 0.03 * h
fig.add_artist(plt.Rectangle((x0-xpad, y0-ypad), w+2*xpad, h+2*ypad, edgecolor='dimgray', linewidth=0.5, fill=False));


NameError: name 'df' is not defined